In [ ]:
from pprint import pprint

dx = 0.001
dy = 0.001
n = 10

def get_size(dx, dy, n):
    return {
        'L_mot' : 5*dx,
        'l_mot' : dy,
        'L_1' :  2*dx,
        'l_1' : dy,
        'L_2' : 4*dx,
        'l_2' : n*dy,
        'delta_vert_12' : dy,
        'L_vacuum' : 5*dx,
        'l_vacuum' : dy,
        'res_x': 2*(n+4),
        'res_y': 5
    }

pprint(get_size(dx,dy,n))

In [ ]:
from main import simulate, processing_only

system_cfg_path = ['cfg_files/square.cfg','cfg_files/square_2x.cfg','cfg_files/square__5y.cfg']
simulation_cfg_path = ['cfg_files/simulation.cfg']*len(system_cfg_path)
processing_simulation_path = ['cfg_files/processing.cfg','cfg_files/processing_2x.cfg','cfg_files/processing__5y.cfg']

for system, simu, proc in zip(system_cfg_path, simulation_cfg_path, processing_simulation_path):
    simulate(system, simu, proc)
    #processing_only(system, simu, proc)


In [ ]:
from main import simulate
simulate('cfg_files/square_.cfg', 'cfg_files/simulation.cfg', 'cfg_files/processing_.cfg')

In [ ]:
from pprint import pprint
# importing functions and modules useful for launching simulations
from main import get_options, get_min_mean_free_path, init_particles_in_system
from main import square, thruster
from main import CollisionHandler, DSMC
from main import scipy_integrate_solve_ivp, rk4, euler_explicit
from main import DataAnalyser, DataSaver
from main import available_particles
from main import post_processing
from main import convert_list_to_string
from collections import OrderedDict

In [ ]:
debug = False

In [ ]:
system_cfg = 'cfg_files/square.cfg' # square, thruster, 
simulation_cfg = 'cfg_files/simulation.cfg'
processing_cfg = 'cfg_files/processing_.cfg'

list_cfg_files = [system_cfg, simulation_cfg, processing_cfg]

cfg_path_dict = {
    'system':system_cfg,
    'simulation':simulation_cfg,
    'processing':processing_cfg
}

In [ ]:
options = get_options(cfg_path_dict)
if(debug): pprint(options)

In [ ]:
processing_only = True
if(processing_only):
    post_processing(options['processing'])

## Building system

In [ ]:
types = options['system']['particles_types']
charges = [available_particles[type_]['charge'] for type_ in types]
masses = [available_particles[type_]['mass'] for type_ in types]
effective_diameters = [available_particles[type_]['effective diameter'] for type_ in types]

min_mean_free_path = get_min_mean_free_path(effective_diameters = effective_diameters, particles_densities = options['system']['particles_densities'])

if(debug): 
    print('Types : {}'.format(types))
    print('Charges in C : {}'.format(charges))
    print('Masses in kg : {}'.format(masses))
    print('Effective diameters in m : {}'.format(effective_diameters))
    print('Min mean free path : {} m.'.format(min_mean_free_path))

In [ ]:
system_type = options['system']['system_type']

if(system_type=='square'):
    system = square(options['system'], min_mean_free_path)
    zone = None
    offset = [0,0]
    
elif(system_type=='thruster'):
    system = thruster(options['system'], min_mean_free_path)
    zone = system.get_zone()
    offset = system.get_offset()

## Initializing particles

In [ ]:
lx, ly, lz = system.get_size()
res_x, res_y = system.get_space_resolutions()
#flux_in, flux_out = 
if(debug):
    print('Space size : {}'.format(system.get_size()))
    print('Resolution : {}'.format(system.get_space_resolutions()))
list_particles, vmean_exp = init_particles_in_system(**options['system'], space_size = [lx,ly], space_resolution=[res_x, res_y], flux = [], zone = zone, offsets = offset, verbose = False, debug = False)

if(vmean_exp==0):
    vmean_exp = 300
# adding particles to the grid
for particle in list_particles:
    system.add(particle)

    # TODO : do as Lui said - don't save it this way. (its okay for now)
if(system_type=='thruster'):
    system.set_sparsed_space()
    pprint("List of cases in the grid that are considered : {}".format(system.grid.sparsed_space_idx))
if(debug):
    pprint({
        'mean speed':vmean_exp,
        'first particles':[part.to_string() for part in list_particles][:5],
        'walls':system.get_walls()
    })
system.plot()

## Preping simulation

In [ ]:
# update function
import numpy as np

if(system_type=='thruster'):
    from dolfin import Point

    def f(Y,t,m,q,zone,E):
        '''
        Renvoie la dérivée de Y en y (en faisant un bilan des forces notamment) pr être entré dans RK4
        Y=[x, y, z, vx, vy, vz] ce n'est pas le Y de liste_Y
        '''
        Ex, Ey = E.split(deepcopy=True)
        vx=Y[3]
        vy=Y[4]
        vz=Y[5]
        if zone.inside(Point(Y[0],Y[1])):
            ax = (1/m) * q * Ex(Y[0], Y[1])
            ay = (1/m) * q * Ey(Y[0], Y[1])
        else :
            ax = 0  #utile si les ki sont hors du mesh,
            ay = 0
        az=0
        return np.array([vx, vy, vz, ax, ay, az])
    
    args = [system.get_zone(), system.get_E()]
    
elif(system_type=='square'):
    def f(Y,t,m,q):
        vx=Y[3]
        vy=Y[4]
        vz=Y[5]

        ax = 0 
        ay = 0
        az = 0
        return np.array([vx, vy, vz, ax, ay, az])
    
    args = []

# DSMC params
N_particles_real = [int(density*lx*ly*lz) for density in options['system']['particles_densities']] # this is the REAL number of particles
N_particles_simu = [int(nb*res_x*res_y) for nb in options['system']['particles_mean_number_per_cell']]
Ne = [i/j for i, j in zip(N_particles_real,N_particles_simu)]

# TODO : see the adaptations for the DMSC with several species of particles

mean_speed = vmean_exp
DSMC_params = {
    'vr_max' : 2*mean_speed,
    'effective_diameter':  effective_diameters[0], # for now
    'Ne' : Ne[0], # this is the number of real particles one simulated particle represents.
    'cell_volume' : lx*ly*lz/(res_x*res_y), # since we are in 2D I don't really know what to add here actually... For now, I add the 3rd dimension rough size, that is l3
    'mean_particles_number_per_cell': sum(options['system']['particles_mean_number_per_cell']) # we don't account for the type
}

if(debug):
    pprint([N_particles_real, N_particles_simu, Ne, DSMC_params])
    
    
use_particles_collisions = False
use_DSMC = True

simu = options['simulation']
if(simu['scheme']=='euler_explicit'):
    integration_scheme = euler_explicit # scipy_integrate_solve_ivp , rk4 , euler_explicit
    # TODO : make a function get_scheme(str)

system.init_dsmc(mean_speed = vmean_exp, integration_scheme = integration_scheme, f = f, f_args = args)
#handler = DSMC(system = system, dsmc_params = DSMC_params, integration_scheme = integration_scheme, f = f)

## Launching Simulation

In [ ]:
from tqdm import tqdm

min_mean_free_time = min_mean_free_path/mean_speed
dt = simu['dt']*min_mean_free_time
t = 0
MAX_INTEGRATION_STEP = simu['number_of_steps']
save_test = options['processing']['save']

In [ ]:
if(save_test):
    saving_period = options['processing']['period']
    dir_saving = options['processing']['path']
    path_saving = dir_saving/(options['processing']['id_test']+'.csv')
    params_dict = OrderedDict()
    
    # general :
    params_dict['id_test']= options['processing']['id_test']
    params_dict['path_to_data'] = str(path_saving) # absolute path

    # on results of the simulation
    params_dict['number_of_collisions'] = '0'
    params_dict['mean_acceptance_rate'] = '0'
    params_dict['mean_proba'] = '0'
    params_dict['mean_vr_norm'] = '0'
    # on system =
    params_dict['nb_cells'] = str(system.get_number_of_cells())
    params_dict['volume'] = str(lx*ly*lz)
    params_dict['x_min'] = str(offset[0])
    params_dict['x_max'] = str(offset[0]+lx)
    params_dict['y_min'] = str(offset[1])
    params_dict['y_max'] = str(offset[1]+ly)
    # on particles =
    params_dict['total_number_of_particles'] = str(system.get_number_of_particles())
    params_dict['particles_types'] = convert_list_to_string(types)
    params_dict['particles_charges'] = convert_list_to_string(charges)
    params_dict['particles_masses'] = convert_list_to_string(masses)
    params_dict['particles_effective_diameters'] = convert_list_to_string(effective_diameters)
    params_dict['particles_densities'] = convert_list_to_string(options['system']['particles_densities']),
    params_dict['nb_parts_per_type'] = convert_list_to_string(N_particles_simu)
    params_dict['nb_parts_real_per_type'] = convert_list_to_string(N_particles_real)
    params_dict['Ne_per_type'] = convert_list_to_string(Ne)
    params_dict['mean_particles_number_per_cell_per_type'] = convert_list_to_string(options['system']['particles_mean_number_per_cell'])
        # deduction
    params_dict['min_mean_free_path'] = str(min_mean_free_path)
    params_dict['min_mean_free_time'] = str(min_mean_free_time)
        # on speed
    params_dict['speed_init_type'] = convert_list_to_string(options['system']['speed_type'])
    params_dict['v_mean'] = str(vmean_exp)
    params_dict['vr_max_init'] = str(DSMC_params['vr_max'])
    params_dict['vr_max_final'] = '0'

    # on simulation params =
    params_dict['dt'] = str(dt)
    params_dict['MAX_INTEGRATION_STEP'] = str(MAX_INTEGRATION_STEP)
    params_dict['integration_scheme'] = integration_scheme.__name__
    #'eta'= eta
    #'loss_charge_proba' = p
        
    # on saving params
    params_dict['saving_period'] = str(saving_period) 
                
    if(debug): pprint(params_dict)
    data_analyser = DataSaver(name_test = options['processing']['id_test']+'.csv', saving_directory = dir_saving)
    data_analyser.save_test_params(dir_saving/'params.csv', params_dict, use_saving_directory = True)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def draw_hist_distribution(system, dt, max_int, path, interval = 200, save_animation = True): # interval : time in ms
    
    def update(num, dt):
        print("Iteration {} - nb of particles : {}".format(num, system.get_number_of_particles()))
        plt.cla() # clf
        t = dt*num
        # make several steps - no need to plot all of them
        system.step(dt, t)
        system.plot()

    fig, ax = plt.subplots(dpi=300, tight_layout=True) # figsize=(3,20)
    #ax.set_aspect('equal', 'box')        
    
    system.plot()
    anim = animation.FuncAnimation(fig, update, interval=interval, frames=max_int, fargs=(dt, ), save_count=max_int)

    if(save_animation):
        anim.save(path/'system_evolution.mp4', dpi=300)
    else:
        plt.show()

In [ ]:
print(system.flux_params)

In [ ]:
#draw_hist_distribution(system, dt, MAX_INTEGRATION_STEP, path = options['processing']['path'], interval = 200)

In [ ]:

handler = system.dsmc
for k in tqdm(range(MAX_INTEGRATION_STEP)): #
    if(debug): print("\nStep {} over {}...\n".format(k+1, MAX_INTEGRATION_STEP))
    system.step(dt, t)
    t+=dt
    list_particles = system.get_list_particles()
    if(save_test and k%saving_period==0 or (k == MAX_INTEGRATION_STEP-1 and k%saving_period!=0)):
        if(len(list_particles) > 0):
            data_analyser.save_everything_to_one_csv(list_particles = list_particles, iteration = k, erase = True)
        handler.save_collisions_matrix(name = dir_saving/("test_"+params_dict['id_test']+"_collision_matrix.txt"), iteration = k)
        
# last saving
number_of_collisions = handler.get_collisions_count()
mean_acceptance_rate = handler.get_acceptance_rate()
mean_vr_norm = handler.get_mean_vr_norm()
vr_max_final = handler.get_vr_norm()
mean_proba = handler.get_mean_proba()

# saving again to the csv.
params_dict['mean_proba']=str(mean_proba)
params_dict['number_of_collisions']=str(number_of_collisions)
params_dict['mean_acceptance_rate']=str(mean_acceptance_rate)
params_dict['mean_vr_norm']=str(mean_vr_norm)
params_dict['vr_max']=str(vr_max_final)
if(debug): pprint(params_dict)
data_analyser.update_saved_params(dir_saving/'params.csv', params_dict, use_saving_directory = False)

In [ ]:
# post processing
#if(save_test): # before starting simulation
post_processing(options['processing'])

In [ ]:
from shutil import copyfile, copy 
from os import mkdir
if(save_test):
    for file in list_cfg_files:
        copy(src = file, dst = dir_saving/"cfg_files/")